# Web Scraping of Job Posting from Indeed website:
Here we have tried to web scrap from Indeed website for 2 locations USA and Canada with different job search keywords like Google cloud, AWS cloud, Azure Cloud, Google for education, G Suite(google workspace) and others.

We have changed the input as text input for different searches and locations. We have also added URLs for navigation purposes(checking point).

1. Programming Language used: Python
2. Libraries used: Beautiful soup, requests
3. Jupyter Notebooks

## Results:
We have extracted 1071 results for Google cloud, 1152 reults for AWS cloud for USA locations (keeps on changing/dependent on the website), saved in results_Indeed.csv file. 
Just like this we can extract results for other job search Keywords.

In [3]:
#!pip install bs4
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [4]:
template = 'https://www.indeed.com/jobs?q={}&l={}'

In [5]:
def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

In [6]:
x= input("Enter job profile:")
y= input("location: ")
url = get_url(x, y)
print(url)

Enter job profile: google cloud
location:  usa


https://www.indeed.com/jobs?q=google+cloud&l=usa


In [7]:
response = requests.get(url)
response

<Response [200]>

In [8]:
soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
cards = soup.find_all('div', 'jobsearch-SerpJobCard')
len(cards)

15

In [10]:
card = cards[1]

In [11]:
atag=card.h2.a

In [12]:
atag.get('title')

'Jr Cloud Practitioner'

In [13]:
company = card.find('span','company').text.strip()

In [14]:
job_title = card.h2.a.get('title')
job_title

'Jr Cloud Practitioner'

In [15]:
job_location = card.find('div','recJobLoc').get('data-rc-loc')
job_location

'Remote'

In [16]:
def get_record(card):
    "extract job from single record"
    atag=card.h2.a
    job_title=atag.get('title')
    company = card.find('span','company').text.strip()
    job_location = card.find('div','recJobLoc').get('data-rc-loc') 
     
    
    record = (job_title,company,job_location)
    
    return record

In [17]:
records = []
for card in cards:
    record = get_record(card)
    records.append(record)

In [18]:
records[0]

('Google Cloud Developer', 'Tekskills Inc', 'Southlake, TX')

In [19]:
soup.find('a', {'aria-label' : 'Next'}).get('href')

'/jobs?q=google+cloud&l=usa&start=10'

In [20]:
while True:
    try:
        url = 'https://www.indeed.com' +soup.find('a',{'aria-label' : 'Next'}).get('href')
    except AttributeError:
        break
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')
    
    for card in cards:
        record = get_record (card)
        records.append(record)

In [21]:
len(records)

1054

In [22]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url
def get_record(card):
    "extract job from single record"
    atag=card.h2.a
    job_title=atag.get('title')
    company = card.find('span','company').text.strip()
    job_location = card.find('div','recJobLoc').get('data-rc-loc') 
      
    
    record = (job_title,company,job_location)
    
    return record

def main(position,location):
    """run main program"""
    records = []
    url = get_url(position,location)
    
    #extract the job data#
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')

        for card in cards:
            record = get_record (card)
            records.append(record)
        try:
            url = 'https://www.indeed.com' +soup.find('a',{'aria-label' : 'Next'}).get('href')
        except AttributeError:
            break
      #saving the job data#
    with open('results_Indeed.csv','w',newline='',encoding ='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['job_title','company','job_location'])
        writer.writerows(records)


In [23]:
main(x, y)